# Binding detailed example: Minuit2

Let's try a non-trivial example: Minuit2 (6.14.0 standalone edition)

### Requirements

* Minuit2 standalone edition (included)

### Expectations
* Be able to minimize a very simple function and get some parameters

In [ ]:
import sys

# Step 1: Get source

* Download Minuit2 source (already provided in `minuit2`)

# Step 2: Plan interface

* You should know what the C++ looks like, and know what you want the Python to look like
* For now, let's replicate the C++ experience

For example: a simple minimizer for $f(x) = x^2$ (should quickly find 0 as minimum):

* Define FCN
* Setup parameters
* Minimize
* Print result

Will use print out for illustration (instead of `MnPrint::SetLevel`)

In [ ]:
%%writefile SimpleFCN.h
#pragma once

#include <Minuit2/FCNBase.h>
#include <Minuit2/FunctionMinimum.h>
#include <Minuit2/MnPrint.h>
#include <Minuit2/MnMigrad.h>

#include <iostream>

using namespace ROOT::Minuit2;

class SimpleFCN : public FCNBase {
    // Always 0.5 for these sorts of fits
    
    double Up() const override {return 0.5;}
    
    // This computes whatever you are going to minimize
    double operator()(const std::vector<double> &v) const override {
        std::cout << "val = " << v.at(0) << std::endl;
        return v.at(0)*v.at(0);
    }
};

In [ ]:
%%writefile simpleminuit.cpp
#include "SimpleFCN.h"

int main() {
    SimpleFCN fcn;
    MnUserParameters upar;
    upar.Add("x", 1., 0.1);
    MnMigrad migrad(fcn, upar);
    FunctionMinimum min = migrad();
    std::cout << min << std::endl;
}

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.18...3.25)
project(Minuit2SimpleExamle LANGUAGES CXX)

include(FetchContent)
FetchContent_Declare(
  minuit2
  URL           https://github.com/root-project/root/archive/refs/tags/v6-26-10.tar.gz
  URL_HASH      SHA256=a84ed095252614c6e2084987fce9ce4a5a62057bfd5a4a2725123ca9f60f674f
  SOURCE_SUBDIR math/minuit2
  DOWNLOAD_NO_PROGRESS
)
FetchContent_MakeAvailable(minuit2)

add_executable(simpleminuit simpleminuit.cpp SimpleFCN.h)
target_link_libraries(simpleminuit PRIVATE Minuit2::Minuit2)

Standard CMake configure and build (using Ninja instead of Make for speed)

In [ ]:
!cmake -GNinja -S . -B build
!cmake --build build

In [ ]:
!./build/simpleminuit

## Step 3: Bind parts we need

* subclassable FCNBase
* MnUserParameters (constructor and `Add(string, double, double)`)
* MnMigrad (constructor and operator())
* FunctionMinimum (cout)


In [ ]:
%mkdir src

## Recommended structure of a Pybind11 program

#### main.cpp

* Builds module
* Avoids imports (fast compile)

```cpp
include <pybind11/pybind11.h>
namespace py = pybind11;

void init_part1(py::module &);
void init_part2(py::module &);

PYBIND11_MODULE(mymodule, m) {
    m.doc() = "Real code would never have such poor documentation...";
    init_part1(m);
    init_part2(m);
}
```

In [ ]:
%%writefile src/pyminuit2.cpp

#include <pybind11/pybind11.h>

namespace py = pybind11;

void init_FCNBase(py::module &);
void init_MnUserParameters(py::module &);
void init_MnMigrad(py::module &);
void init_FunctionMinimum(py::module &);

PYBIND11_MODULE(minuit2, m) {
    init_FCNBase(m);
    init_MnUserParameters(m);
    init_MnMigrad(m);
    init_FunctionMinimum(m);
}

We will put all headers in a collective header (not a good idea unless you are trying to show files one per slide).

In [ ]:
%%writefile src/PyHeader.h
#pragma once
#include <pybind11/pybind11.h>
#include <pybind11/functional.h>
#include <pybind11/numpy.h>
#include <pybind11/stl.h>

#include <Minuit2/FCNBase.h>
#include <Minuit2/MnMigrad.h>
#include <Minuit2/MnApplication.h>
#include <Minuit2/MnUserParameters.h>
#include <Minuit2/FunctionMinimum.h>

namespace py = pybind11;
using namespace pybind11::literals;
using namespace ROOT::Minuit2;

## Overloads

* Pure virtual methods cannot be instantiated in C++
* Have to provide "Trampoline class" to provide Python class

In [ ]:
%%writefile src/FCNBase.cpp
#include "PyHeader.h"
class PyFCNBase : public FCNBase {
   public:
     using FCNBase::FCNBase;

     double operator()(const std::vector<double> &v) const override {
         PYBIND11_OVERLOAD_PURE_NAME(
             double, FCNBase, "__call__", operator(), v);}

     double Up() const override {
         PYBIND11_OVERLOAD_PURE(double, FCNBase, Up, );}
 };
void init_FCNBase(py::module &m) {
    py::class_<FCNBase, PyFCNBase>(m, "FCNBase")
         .def(py::init<>())
         .def("__call__", &FCNBase::operator())
         .def("Up", &FCNBase::Up);
}

## Overloaded function signatures:
* C++11 syntax: `(bool (MnUserParameters::*)(const std::string &, double)) &MnUserParameters::Add`
* C++14 syntax: `py::overload_cast<const std::string &, double>(&MnUserParameters::Add)`

In [ ]:
%%writefile src/MnUserParameters.cpp
#include "PyHeader.h"

void init_MnUserParameters(py::module &m) {
    py::class_<MnUserParameters>(m, "MnUserParameters")
        .def(py::init<>())
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double)) &MnUserParameters::Add)
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double, double)) &MnUserParameters::Add)
    ;
}

## Adding default arguments (and named arguments)
* Using `""_a` literal, names and even defaults can be added 

In [ ]:
%%writefile src/MnMigrad.cpp
#include "PyHeader.h"
        
void init_MnMigrad(py::module &m) {
    py::class_<MnApplication>(m, "MnApplication")
        .def("__call__",
             &MnApplication::operator(),
             "Minimize the function, returns a function minimum",
             py::arg("maxfcn")    = 0,
             "tolerance"_a = 0.1);
    
    py::class_<MnMigrad, MnApplication>(m, "MnMigrad")
        .def(py::init<const FCNBase &, const MnUserParameters &, unsigned int>(),
             "fcn"_a, "par"_a, "stra"_a = 1)
    ;
}    

## Lambda functions

* Pybind11 accepts lambda functions, as well

In [ ]:
%%writefile src/FunctionMinimum.cpp
#include "PyHeader.h"

#include <sstream>
#include <Minuit2/MnPrint.h>

void init_FunctionMinimum(py::module &m) {
    py::class_<FunctionMinimum>(m, "FunctionMinimum")
        .def("__str__", [](const FunctionMinimum &self) {
            std::stringstream os;
            os << self;
            return os.str();
        })
    ;
}

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.18...3.25)
project(Minuit2SimpleExamle LANGUAGES CXX)

set(CMAKE_POSITION_INDEPENDENT_CODE ON)

include(FetchContent)
FetchContent_Declare(
  minuit2
  URL           https://github.com/root-project/root/archive/refs/tags/v6-26-10.tar.gz
  URL_HASH      SHA256=a84ed095252614c6e2084987fce9ce4a5a62057bfd5a4a2725123ca9f60f674f
  SOURCE_SUBDIR math/minuit2
  DOWNLOAD_NO_PROGRESS
  EXCLUDE_FROM_ALL
)
FetchContent_MakeAvailable(minuit2)

find_package(pybind11)

file(GLOB OUTPUT src/*.cpp)
pybind11_add_module(minuit2 ${OUTPUT})
target_link_libraries(minuit2 PUBLIC Minuit2::Minuit2)

install(TARGETS minuit2 DESTINATION ${SKBUILD_PLATLIB_DIR})

We can use this CMakeLists from scikit-build-core:

In [ ]:
%%writefile pyproject.toml

[build-system]
requires = ["scikit-build-core>=0.1.2", "pybind11>=2.10"]
build-backend = "scikit_build_core.build"

[project]
name = "minuit2"
version = "6.26.10"

[tool.scikit-build]
experimental = true
wheel.install-dir = "/null"

WARNING: Without the experimental redirection above, this installs the minuit2 headers and static lib into the wrong place in the wheel (they really shouldn't be in the wheel at all). But that's okay, we aren't going to use this wheel.

In [ ]:
!{sys.executable} -m pip wheel . -v

# Usage

We can now use our module! We'll just grab the file we need out of the wheel instead of installing it:

In [ ]:
import zipfile
from pathlib import Path
import shutil

with zipfile.ZipFile(*Path.cwd().glob("minuit2-6.26.10-*.whl")) as f:
    for member in f.filelist:
        print(member.filename)
        if member.filename.startswith("minuit2."):
            f.extract(member, ".")

In [ ]:
import minuit2


class SimpleFCN(minuit2.FCNBase):
    def Up(self):
        return 0.5

    def __call__(self, v):
        print("val =", v[0])
        return v[0] ** 2

In [ ]:
fcn = SimpleFCN()
upar = minuit2.MnUserParameters()
upar.Add("x", 1.0, 0.1)
migrad = minuit2.MnMigrad(fcn, upar)
minimum = migrad()

In [ ]:
print(minimum)

# Done

* See [GooFit's built in Minuit2 bindings](https://github.com/GooFit/GooFit/tree/master/python/Minuit2) for a more complete example
* Pybind11 bindings can talk to each other at the C level!